# Imports

In [2]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import PyPDF2
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import cloudinary.uploader
import json

# Loading current banks list

In [23]:
post_req = {"1":{"url":"https://www.abanca.pt", 
                    "name":"ABANCA Corporación Bancaria, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0170"}
             ,
              "2":{"url":"https://www.bankinter.pt", 
                    "name":"Bankinter, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0269"}
              ,
              "3":{"url":"https://www.bancobic.ao", 
                    "name":"Banco BIC, SA", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0079"}
                ,
                "4":{"url":"https://www.bancobai.ao", 
                    "name":"BAI - Banco Angolano de Investimentos.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0008"}
              }
post_req = json.dumps(post_req)
post_req

'{"1": {"url": "https://www.abanca.pt", "name": "ABANCA Corporaci\\u00f3n Bancaria, S.A.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0170"}, "2": {"url": "https://www.bankinter.pt", "name": "Bankinter, S.A.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0269"}, "3": {"url": "https://www.bancobic.ao", "name": "Banco BIC, SA", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0079"}, "4": {"url": "https://www.bancobai.ao", "name": "BAI - Banco Angolano de Investimentos.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0008"}}'

In [15]:
url = 'http://127.0.0.1:5000/'
do = 'dopdfs'
retrieve = 'retrievepdfs'

In [17]:
# do pdfs
r = requests.post(url+do, data=post_req)
r

<Response [500]>

In [27]:
# collect them // wait a couple of minutes
banks_r = requests.get(url+retrieve)
banks = banks_r.content

In [167]:
banks_dict = json.loads(banks)

In [168]:
banks_dict

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': 320,
  'url': 'https://www.abanca.pt'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Com

# interate through given filenames

In [169]:
# what is on the index page? 
## ["FOLHETO DE COMISSÕES E DESPESAS", "1.1"]

In [173]:
def file_decrypt(pdf_url, filename="tempe.pdf"):
    """Accepts pdf urls. Returns PyPDF2 FileReader Objects"""
    print(f'file_decrypt was called for {pdf_url}')
    response = requests.get(pdf_url)
    if response.status_code == 200:
        temp = open(filename, "wb")
        temp.write(response.content)
        temp.close()
        command="cp "+filename+" temp.pdf; qpdf --password='' --decrypt temp.pdf "+filename
        os.system(command)
        print('File decrypted (qpdf)')
        #re-open the decrypted file
        pdfFile = PdfFileReader(filename)
        os.remove(filename)
    else: 
        print(f'{url} could not be reached with file_decrypt. Response: {response}')
        return None
    return pdfFile

In [174]:
searchlist = ["Clientes Particulares", "1.1"]

def get_index_pg(searchlist=searchlist):
    for id, values in banks_dict.items():
        print(f'handling pdfs from {values.get("url")}')
        # defining the URLs in a list
        web_pdfs = values.get("list_pdfs")
        web_pdfs.insert(0, values.get("bp_pdf_url"))
        # trying to remotely parse the pdfs all pdf pages
        for pdf_url in web_pdfs:
            try:
                remote = urlopen(Request(pdf_url)).read()
                memory = BytesIO(remote)
                pdf_file = PdfFileReader(memory)
                # check file encryption - if yes, call decrypter
                if pdf_file.isEncrypted:
                    print(f"file {pdf_url} is encrypted. \nstarting to decrypt")
                    ## pass in the memory object or path string
                    pdf_file = file_decrypt(pdf_url)  
                else:
                    print(f"file {pdf_url} is not encrypted. \ncontinuing...")
                    pdf_file = pdf_file
                # iterate through every page in one file to find index page
                for i,page in enumerate(pdf_file.pages):
                    validator = [x in page.extractText() for x in searchlist]
#                     print(f'validator: {validator}')
                    if all(validator):
                        print(f'found all terms of {searchlist} on page {i+1} of file {pdf_url}')
                        writer = PdfFileWriter()
                        writer.addPage(page)
                        out_path = f'output_{id}.pdf'
                        outfile = open(out_path, 'wb')
                        writer.write(outfile)
                        outfile.close()
                        print(f'saved index page to {out_path}')
                        memory.close
                        break
            except Exception as e:
                print(f"url not found. Error: {e}, url: {pdf_url}")
                continue
            break


In [175]:
get_index_pg()

handling pdfs from https://www.abanca.pt
file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf is encrypted. 
starting to decrypt
file_decrypt was called for https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
File decrypted (qpdf)
found all terms of ['Clientes Particulares', '1.1'] on page 2 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
saved index page to output_1.pdf
handling pdfs from https://www.bankinter.pt
file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf is encrypted. 
starting to decrypt
file_decrypt was called for https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
File decrypted (qpdf)
found all terms of ['Clientes Particulares', '1.1'] on page 2 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
saved index page to output_2.pdf
handling pdfs from 

# PyPDF2

In [ ]:
## fpr remote files
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)
writer = PdfFileWriter()

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [ ]:
path = 'bic_all_prices.pdf'

In [ ]:
pdf = PdfFileReader(path)

In [ ]:
def get_index_page(path, search):    
    p = open(path, "rb")
    pdf = PdfFileReader(p)
    page_nums = []
    for page in pdf.pages:
        if search in page.extractText():
            page_nums.append(pdf.getPageNumber(page))
    return page_nums

# Trying other libraries

## PDFX

In [4]:
# import module
import pdfx
  
# reading pdf file
pdf = pdfx.PDFx("https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf")
  
# display
print(pdf)

## PDF Miner

In [5]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [6]:
path = 'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf'

In [7]:
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
# pdfFile = PdfFileReader(memoryFile)

convert_pdf_to_txt(r.content)

OSError: [Errno 36] File name too long: b'{\n  "abanca": {\n    "pb_bank_id": "0170", \n    "pb_bank_pdf": "https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf", \n    "pdfs": [\n      "https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf", \n      "https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf", \n      "https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf", \n      "https://www.abanca.pt/files/documents/glossario-6479e89f.pdf", \n      "https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf"\n    ], \n    "pricelist_url": "https://www.abanca.pt/pt/precario/", \n    "url": "https://www.abanca.pt"\n  }, \n  "bankinter": {\n    "pb_bank_id": "0269", \n    "pb_bank_pdf": "https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf", \n    "pdfs": [\n      "https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TFS

In [8]:
## fpr remote files
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)
writer = PdfFileWriter()

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [37]:
path = 'bic_all_prices.pdf'

In [48]:
pdf = PdfFileReader(path)

In [50]:
pdf.content

AttributeError: 'PdfFileReader' object has no attribute 'content'

In [43]:
def get_rel_page(path, search):    
    p = open(path, "rb")
    pdf = PdfFileReader(p)
    page_nums = []
    for page in pdf.pages:
        if search in page.extractText():
            page_nums.append(pdf.getPageNumber(page))
    return page_nums


In [45]:
regex = r' '
get_rel_page(path, regex)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77]